In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!wget https://storage.googleapis.com/kaggle-data-sets/107620/256618/bundle/archive.zip

In [ ]:
import os, shutil, numpy as np, cv2,zipfile
import subprocess
from google.colab import files
from IPython.display import Audio
import librosa
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow

In [ ]:
from tensorflow.keras import Sequential

In [ ]:
from tensorflow.keras import layers 

In [ ]:
u=files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio

 98% 419M/429M [00:07<00:00, 50.2MB/s]
100% 429M/429M [00:07<00:00, 63.1MB/s]


In [ ]:
ls

drive/  kaggle.json  ravdess-emotional-speech-audio.zip  sample_data/


In [ ]:
!unzip './drive/My Drive/ravdess-emotional-speech-audio.zip'

In [ ]:
ls

Actor_01/  Actor_07/  Actor_13/  Actor_19/  audio_speech_actors_01-24/
Actor_02/  Actor_08/  Actor_14/  Actor_20/  drive/
Actor_03/  Actor_09/  Actor_15/  Actor_21/  kaggle.json
Actor_04/  Actor_10/  Actor_16/  Actor_22/  ravdess-emotional-speech-audio.zip
Actor_05/  Actor_11/  Actor_17/  Actor_23/  sample_data/
Actor_06/  Actor_12/  Actor_18/  Actor_24/


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=True)

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
for i in range(1,9):
    os.mkdir(str(i))

In [ ]:
for act in range(1,21):
    for i in os.listdir('Actor_'+str(act).zfill(2)+'/'):
        
        doc_name=i
        n=doc_name.split('.wav')
        w=int(n[0].split('-')[0])#01 = full-AV, 02 = video-only, 03 = audio-only
        labz=int(n[0].split('-')[2])#01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
              
        shutil.move('Actor_'+str(act).zfill(2)+'/'+i,str(labz)+'/Actor_'+str(act).zfill(2)+'_'+i)

In [ ]:
n,w,labz

In [ ]:
ls

1/  Actor_01/  Actor_09/  Actor_17/  audio_speech_actors_01-24/
2/  Actor_02/  Actor_10/  Actor_18/  drive/
3/  Actor_03/  Actor_11/  Actor_19/  kaggle.json
4/  Actor_04/  Actor_12/  Actor_20/  ravdess-emotional-speech-audio.zip
5/  Actor_05/  Actor_13/  Actor_21/  sample_data/
6/  Actor_06/  Actor_14/  Actor_22/
7/  Actor_07/  Actor_15/  Actor_23/
8/  Actor_08/  Actor_16/  Actor_24/


In [ ]:
arr2,lab2=[],[]
for i in range(1,9):
    #arr=[]
    for j in os.listdir(str(i)):
        x,sr=librosa.load(str(i)+'/'+j, sr=44100)
        x2=librosa.to_mono(x)
        diff=44100*4-x2.shape[0]
        
        if diff>0:
            x2=np.concatenate((np.zeros(diff)+0.0000002,x2),axis=0)
        if diff<0:
            diff= abs(diff)
            if diff%2==0:
                x2=np.array(x2[int(diff/2):x2.shape[0]-int(diff/2)])
            if diff%2==1:
                x2=np.array(x2[int(diff/2)+1:x2.shape[0]-int(diff/2)])
        arr2.append(librosa.feature.mfcc(x2, sr=44100, n_mfcc=20))
        #arr1.append(librosa.feature.mfcc(x2, sr=22050, n_mfcc=44))
        lab2.append(i)

In [ ]:
arr1,lab=[],[]
for i in range(1,9):
    #arr=[]
    for j in os.listdir('./drive/My Drive/'+str(i)):
        
        x,sr=librosa.load('./drive/My Drive/'+str(i)+'/'+j, sr=44100)

        x2=librosa.to_mono(x)
        diff=44100*4-x2.shape[0]
        
        if diff>0:
            x2=np.concatenate((np.zeros(diff)+0.0000002,x2),axis=0)
        if diff<0:
            diff=abs(diff)
            if diff%2==0:
                x2=np.array(x2[int(diff/2):x2.shape[0]-int(diff/2)])
            if diff%2==1:
                x2=np.array(x2[int(diff/2)+1:x2.shape[0]-int(diff/2)])
        
        arr1.append(librosa.feature.mfcc(x2, sr=44100, n_mfcc=20))
        #arr1.append(librosa.feature.mfcc(x2, sr=22050, n_mfcc=44))
        lab.append(i)

In [ ]:
arr1shapes=[]
for i in arr1:
    if i.shape not in arr1shapes:
        arr1shapes.append(i.shape)

In [ ]:
arr1shapes=[]
for i in range(1,9):
    #arr=[]
    for j in os.listdir('./drive/My Drive/'+str(i)):
        
        x,sr=librosa.load('./drive/My Drive/'+str(i)+'/'+j, sr=44100)
        
        x2=librosa.to_mono(x)
        if x2.shape not in arr1shapes:
            arr1shapes.append(x2.shape)
        #diff=44100*4-x2.shape[0]

In [ ]:
arr1shapes

In [ ]:
arr1= np.array(arr1)
lab=np.array(lab)
arr1.shape, lab.shape

((1200, 20, 345), (1200,))

In [ ]:
arr2= np.array(arr2)
lab2=np.array(lab2)
arr2.shape, lab2.shape

((1200, 20, 345), (1200,))

In [ ]:
lab2

array([1, 1, 1, ..., 8, 8, 8])

In [ ]:
data= np.concatenate((arr1,arr2),axis=0)
label=np.concatenate((lab,lab2),axis=0)

In [ ]:
data.shape, label.shape

((2400, 20, 345), (2400,))

In [ ]:
data= data.reshape(len(data), 345,20)

In [ ]:
from sklearn.utils import shuffle

In [ ]:
data,label2 = shuffle(data, label2, random_state=0)

In [ ]:
# label= label-1
# label2= to_categorical(label)

In [ ]:
# Audio(x2,rate=44100)#'./drive/My Drive/'+str(i)+'/'+j)

In [ ]:
data

Our Model

In [ ]:
model = Sequential()
model.add(layers.Conv1D(256, 2,padding='same',input_shape=(345,20)))
model.add(layers.Activation('tanh'))

model.add(layers.Conv1D(128, 2,padding='same'))
model.add(layers.Activation('tanh'))

model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv1D(128, 5,padding='same'))
model.add(layers.Activation('tanh'))
model.add(layers.Conv1D(64, 3,padding='same'))
model.add(layers.Activation('tanh'))
model.add(layers.MaxPooling1D(pool_size=(2)))
model.add(layers.Dropout(0.1))

model.add(layers.Flatten())
model.add(layers.Dense(128))
model.add(layers.Dense(64))
model.add(layers.Dense(8))
model.add(layers.Activation('softmax'))

opt = SGD(learning_rate=1e-3)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(data,label2, validation_split=0.15,epochs=100)#, batch_size=200)

Epoch 1/100
64/64 [==============================] - 9s 133ms/step - loss: 2.2076 - accuracy: 0.1397 - val_loss: 2.0265 - val_accuracy: 0.1833
Epoch 2/100
64/64 [==============================] - 8s 127ms/step - loss: 2.1528 - accuracy: 0.1525 - val_loss: 2.0261 - val_accuracy: 0.1806
Epoch 3/100
64/64 [==============================] - 8s 128ms/step - loss: 2.0821 - accuracy: 0.1848 - val_loss: 1.9584 - val_accuracy: 0.2361
Epoch 4/100
64/64 [==============================] - 8s 127ms/step - loss: 2.0822 - accuracy: 0.1725 - val_loss: 1.9378 - val_accuracy: 0.2194
Epoch 5/100
64/64 [==============================] - 8s 127ms/step - loss: 2.0688 - accuracy: 0.1873 - val_loss: 1.9491 - val_accuracy: 0.1833
Epoch 6/100
64/64 [==============================] - 8s 128ms/step - loss: 2.0128 - accuracy: 0.2132 - val_loss: 1.9123 - val_accuracy: 0.2417
Epoch 7/100
64/64 [==============================] - 8s 127ms/step - loss: 2.0010 - accuracy: 0.2265 - val_loss: 1.8755 - val_accuracy: 0.2583

continue training for 100 epochs more

In [ ]:
model.fit(data,label2, validation_split=0.15,epochs=100)#, batch_size=200)

Epoch 1/100
64/64 [==============================] - 8s 130ms/step - loss: 0.9164 - accuracy: 0.6725 - val_loss: 1.1102 - val_accuracy: 0.6028
Epoch 2/100
64/64 [==============================] - 8s 129ms/step - loss: 0.9020 - accuracy: 0.6853 - val_loss: 1.0919 - val_accuracy: 0.6056
Epoch 3/100
64/64 [==============================] - 8s 127ms/step - loss: 0.8940 - accuracy: 0.6819 - val_loss: 1.1091 - val_accuracy: 0.6028
Epoch 4/100
64/64 [==============================] - 8s 128ms/step - loss: 0.8765 - accuracy: 0.6931 - val_loss: 1.1166 - val_accuracy: 0.6028
Epoch 5/100
64/64 [==============================] - 8s 127ms/step - loss: 0.8897 - accuracy: 0.6814 - val_loss: 1.1080 - val_accuracy: 0.6111
Epoch 6/100
64/64 [==============================] - 8s 128ms/step - loss: 0.8948 - accuracy: 0.6833 - val_loss: 1.0964 - val_accuracy: 0.5861
Epoch 7/100
64/64 [==============================] - 8s 128ms/step - loss: 0.8836 - accuracy: 0.6828 - val_loss: 1.0908 - val_accuracy: 0.5889

In [ ]:
!wget https://gist.githubusercontent.com/albertomontesg/d8b21a179c1e6cca0480ebdf292c34d2/raw/1e2620258ce619e18d3219cf962a3f0a0ed17093/sports_1M.json

--2021-08-15 06:41:16--  https://gist.githubusercontent.com/albertomontesg/d8b21a179c1e6cca0480ebdf292c34d2/raw/1e2620258ce619e18d3219cf962a3f0a0ed17093/sports_1M.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10040 (9.8K) [text/plain]
Saving to: ‘sports_1M.json’

sports_1M.json      100%[===================>]   9.80K  --.-KB/s    in 0s      

2021-08-15 06:41:16 (50.7 MB/s) - ‘sports_1M.json’ saved [10040/10040]



This is future work to extract 3D features and merge MFCC audio features with this.

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from keras.optimizers import SGD

def get_model(summary=False):
    """ Return the Keras model of the network
    """
    model = Sequential()
    # 1st layer group
    model.add(Convolution3D(64, 3, 3, 3, activation='relu', 
                            padding='same', name='conv1',
                            subsample=(1, 1, 1)))
                            #input_shape=(3, 16, 112, 112)
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), 
                           padding='valid', name='pool1'))
    # 2nd layer group
    model.add(Convolution3D(128, 3, 3, 3, activation='relu', 
                            padding='same', name='conv2',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool2'))
    # 3rd layer group
    model.add(Convolution3D(256, 3, 3, 3, activation='relu', 
                            padding='same', name='conv3a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(256, 3, 3, 3, activation='relu', 
                            padding='same', name='conv3b',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool3'))
    # 4th layer group
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            padding='same', name='conv4a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            padding='same', name='conv4b',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool4'))
    # 5th layer group
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            padding='same', name='conv5a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            padding='same', name='conv5b',
                            subsample=(1, 1, 1)))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool5'))
    model.add(Flatten())
    # FC layers group
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(.5))
    model.add(Dense(487, activation='softmax', name='fc8'))
    if summary:
        print(model.summary())
    return model

model = get_model(summary=True)

In [ ]:
from keras.models import model_from_json
model = model_from_json(open('sports_1M.json', 'r').read())
model.load_weights('./drive/My Drive/sports1M_weights.h5')
model.compile(loss='mean_squared_error', optimizer='sgd')